# Imports

In [ ]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow_addons as tfa

import pandas as pd
import urllib.request
from urllib.parse import urlparse

import warnings
warnings.simplefilter(action='ignore')

# Load Data

In [ ]:
df_train = pd.read_parquet('../input/movie-posters-2/df_train-split_v1.gzip')
df_val = pd.read_parquet('../input/movie-posters-2/df_eval-split_v1.gzip')
df_test = pd.read_parquet('../input/movie-posters-2/df_test-split_v1.gzip')

def get_path_from_url(url):
    if url is None:
        return None    
    return url.split('/')[-1]

df_train['filename'] = df_train['poster_url'].apply(get_path_from_url)
df_val['filename'] = df_val['poster_url'].apply(get_path_from_url)
df_test['filename'] = df_test['poster_url'].apply(get_path_from_url)

display(df_train.head(2))
display(df_val.head(2))
display(df_test.head(2))

In [ ]:
df_train.shape, df_val.shape, df_test.shape

# Create Image generators for train, validate and test

In [ ]:
IMAGES_DIR = '../input/movie-posters-2/data'

label_cols = [col for col in df_train.columns if col.startswith('x0_')]

In [ ]:
datagen = ImageDataGenerator()

train_generator = datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col=label_cols,
    batch_size=64,
    shuffle=True,
    class_mode="raw",    
    target_size=(299, 299),
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=df_val,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col=label_cols,
    batch_size=64,
    class_mode="raw",
    target_size=(299, 299)
)

test_generator = datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=IMAGES_DIR,
    x_col="filename",
    y_col=label_cols,
    batch_size=64,
    class_mode="raw",
    target_size=(299, 299),
    validate_filenames=True
)

# Create Random Model

In [ ]:
m1 = keras.Sequential()
m1.add(
    layers.Lambda(lambda x: tf.random.uniform((1,19), minval=0.0, maxval=1.0), input_shape=(299,299,3))
    )
m1.compile(optimizer='adam', loss='binary_crossentropy', 
           metrics=['accuracy',
                    tfa.metrics.FBetaScore(num_classes=19, average='micro', threshold=0.5, name='f1_score_micro',),
                    tfa.metrics.FBetaScore(num_classes=19, average='weighted', threshold=0.5, name='f1_score_weighted'),])

In [ ]:
m1.evaluate(test_generator)